In [1]:
import pandas as pd
import numpy as np
import glob
from matplotlib import pyplot as plt
from sklearn import metrics
pd.options.display.max_columns = 1200
pd.options.display.max_rows = 1200

# Read in and Prep Data

In [2]:
df = pd.read_csv('data/autoperf_filtered_1000000ch', header=0).sample(frac=1)

# li = []
# for filename in glob.glob("/home/luckierdodge/repos/jupyter-notebooks/data/autoperf_complete/autoperf-201*.csv"):
#     frame = pd.read_csv(filename, header=0)
#     li.append(frame)
# df = pd.concat(li, axis=0, ignore_index=True)

# df.dropna(inplace=True);

In [3]:
# counts = df.zero_execName.value_counts().to_dict()
# df_filtered = pd.DataFrame()
# for name in counts:
#     if counts[name] >= 500:
#         df_filtered = df_filtered.append(df[df["zero_execName"] == name])
# df = df_filtered.copy()

In [4]:
df.zero_elapsedTime.describe()

count    953803.000000
mean       2484.694284
std        6904.244183
min           0.000409
25%          55.176543
50%         568.214495
75%        2597.903987
max       86150.320240
Name: zero_elapsedTime, dtype: float64

In [5]:
train_size = .8
train = df.head(int(df.size * train_size))
test = df.tail(int(df.size * (1-train_size)))

In [6]:
features = ["zero_execName", "zero_userName"]
# targets = ["zero_elapsedTime"]
targets = ["zero_numRanks"]

X_train = train[features]
X_test = test[features]
Y_train = train[targets]
Y_test = train[targets]

# Linear Regression using SciKit Learn (One-Hot Encoding)

In [7]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=True)
enc.fit(X_train.values)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [8]:
X_train_one_hot = enc.transform(X_train.values)
X_test_one_hot = enc.transform(X_test.values)

In [9]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)
model.fit(X_train_one_hot, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [10]:
model.score(X_test_one_hot, Y_test)

0.4972724355989899

In [11]:
import math
Y_predict = model.predict(X_test_one_hot)
regression_model_mse = metrics.mean_squared_error(Y_predict, Y_test)
math.sqrt(regression_model_mse)

20519.52016717523

In [12]:
print(math.sqrt(regression_model_mse) / (max(Y_test.values) - min(Y_test.values)))

[0.01956896]


# Keras Analysis

In [13]:
import tensorflow as tf

In [14]:
train_examples = X_train_one_hot
train_targets = Y_train
test_examples = X_test_one_hot
test_targets = Y_test

In [15]:
train_targets.shape

(953803, 1)

In [16]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(train_examples.shape[1],)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(64, activation='relu',))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1))

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(optimizer=optimizer,
             loss='mean_squared_error',
             metrics=['mean_absolute_error', 'mean_squared_error'],
             )

Instructions for updating:
Use tf.cast instead.


In [18]:
model.fit(train_examples, train_targets, epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
953803/953803 [==============================] - 88s 92us/sample - loss: 663849366.9262 - mean_absolute_error: 6721.5493 - mean_squared_error: 663849088.0000
Epoch 2/10
953803/953803 [==============================] - 89s 93us/sample - loss: 471916671.3758 - mean_absolute_error: 6140.2339 - mean_squared_error: 471917472.0000
Epoch 3/10
953803/953803 [==============================] - 86s 90us/sample - loss: 453970114.6109 - mean_absolute_error: 6196.8809 - mean_squared_error: 453969280.0000
Epoch 4/10
953803/953803 [==============================] - 85s 89us/sample - loss: 448583565.8573 - mean_absolute_error: 6256.0347 - mean_squared_error: 448582528.0000
Epoch 5/10
953803/953803 [==============================] - 90s 94us/sample - loss: 446950403.3604 - mean_absolute_error: 6260.9321 - mean_squared_error: 446951040.0000
Epoch 6/10
953803/953803 [==============================] - 86s 91us/sample - loss: 444443900.3696 - mean_a

In [19]:
test_loss, test_abs_err, test_sqr_err = model.evaluate(test_examples, test_targets)
# print('Test accuracy:', test_acc)

953803/953803 [==============================] - 60s 63us/sample - loss: 1493486356.7562 - mean_absolute_error: 5767.9385 - mean_squared_error: 1493493760.0000


In [20]:
print(test_abs_err / (max(test_targets.values) - min(test_targets.values)))

[0.00550074]
